In [71]:
filename = "./dataset/forCLFnohash1000.csv"
with open(filename) as f:
    content = f.readlines()

##Remove Labels
del(content[0])

##Remove '\n' elements
content = [x for x in content if x != '\n']

##Striping Whitespaces
content = [x.strip() for x in content]

##Spilting by ','
content = [x.split(',') for x in content]
##print(content)

In [72]:
##docs = [x[0] for x in content]
docs = [x[0] for x in content[0:125]] + [x[0] for x in content[125:250]]
##print(docs)

In [73]:
from pythainlp.corpus import stopwords
from pythainlp.tokenize import word_tokenize
import re

def tokenizer(sentence):
    pattern = re.compile(r"[^\u0E00-\u0E4C]|^'|'$|''|\u0E46|\u0E3F")
    char_to_remove = re.findall(pattern, sentence)
    list_with_char_removed = [char for char in sentence if not char in char_to_remove]
    result_string = ''.join(list_with_char_removed)
    return word_tokenize(result_string, engine='deepcut')

In [123]:
tokenized_docs = [tokenizer(x) for x in docs]
##print(tokenized_docs)

[['นอก', 'จาก', 'จะ', 'คัน', 'ปาก', 'อยาก', 'เม้าท์', 'แล้ว', 'ยัง', 'คัน', 'ไม้', 'คัน', 'มือ', 'รวม', 'ถึง', 'คัน', 'ตีน', 'อยาก', 'กระทืบ', 'คน', 'ด้วย'], ['ดูแบง', 'ไม่', 'ได'], ['รภ', 'แม่ง', 'ติด', 'ไร', 'ว่ะ'], ['ซัก', 'วัน', 'บ้าน', 'จะ', 'แตก', 'เพราะ', 'ปากแม่', 'นั่น', 'แหล'], ['ซัมซุง', 'ไม่', 'มี', 'ช่อง', 'ห้อง', 'กุ๊งกิ๊งหรอย้'], ['สลัด', 'มา', 'กกกกกกจิก', 'ได้', 'จิก', 'ดี', 'หึยยย'], ['กาก', 'ที่สุด', 'อยู่', 'ที่', 'เน็ต', 'ทรู', 'ที่', 'ประมูล', 'คลื่น', 'ความ', 'ถี่', 'มา', 'เยอะ', 'ไม่', 'ได้', 'ช่วย', 'อะไร', 'เลย', 'ที', 'หลัง', 'ไม่', 'ต้อง', 'ไป', 'ประมูล', 'มา', 'ให้', 'เสีย', 'ตัง', 'หรอก', 'เอา', 'เงิน', 'มา', 'พัฒนา', 'ระบบ', 'ดี', 'กว่'], ['โอ้ยแม่ง', 'อย่า', 'ให้', 'กู', 'ใช้', 'สันดาร', 'นะ', 'ละมึง', 'จะ', 'รู้จัก', 'ตัว', 'กู', 'ที่', 'แท้จริง', 'กู', 'ไม่', 'ใช่', 'คน', 'ตลก', 'อย่าง', 'ที่', 'มึง', 'เห็น'], ['อีนุบน', 'ถึง', 'พยาบาล', 'ส่วน', 'ตัว', 'ให้', 'ฟัง', 'อีก', 'แล้'], ['ขัดใจ', 'พี่เค้า', 'ได้', 'ไง', 'ยุ'], ['เพื่อน', 'ลืม', 'นั'], ['เขา'

In [75]:
word_set = [set(x) for x in tokenized_docs]
##print(word_set)

In [164]:
w = word_set[0].union(word_set[1])
for x in range(len(word_set)):
    w = w.union(word_set[x])
##'โมโห' in w

True

In [166]:
wordDictList = []
for doc in tokenized_docs:
    wordDict = dict.fromkeys(w,0)
    for word in doc:
        wordDict[word]+=1
    wordDictList.append(wordDict)
##wordDictList
##'โมโห' in wordDictList[0]

True

In [78]:
import pandas as pd
pd.DataFrame(wordDictList)

,,กกกกก,กกกกกกจิก,กฎหมาย,กทม,กรอบ,กระดาษ,กระตุกก,กระทง,กระทืบ,...,ไส้,ไห,ไหน,ไหม,ไหมม,ไหร่,ไหว,ไอ,ไอพี,ไอ่เด็ก
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [79]:
def computeTF(wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word] = count/float(bowCount)
    return tfDict

In [80]:
tfDict_list = []
for word,doc in list(zip(wordDictList, tokenized_docs)):
    tfDict_list.append(computeTF(word,doc))
##tfDict_list

In [81]:
def computeIdf(docList):
    import math
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(),0)
    for doc in docList:
        for word, val in doc.items():
            if val>0:
                idfDict[word]+=1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log(N/float(val))
        
    return idfDict

In [82]:
idfs = computeIdf(wordDictList)

In [83]:
def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word,val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf

In [84]:
tfidf_list = []
for tfDict in tfDict_list:
    tfidf_list.append(computeTFIDF(tfDict,idfs))
##tfidf_list

In [85]:
import pandas as pd
pd.DataFrame(tfidf_list)

,,กกกกก,กกกกกกจิก,กฎหมาย,กทม,กรอบ,กระดาษ,กระตุกก,กระทง,กระทืบ,...,ไส้,ไห,ไหน,ไหม,ไหมม,ไหร่,ไหว,ไอ,ไอพี,ไอ่เด็ก
0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.22992,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
1,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
2,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
3,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
4,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
5,0.0,0.0,0.78878,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
6,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
7,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
8,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
9,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.00000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0


In [264]:
import numpy as np
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import recall_score, precision_score, f1_score
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler

X = []
for l in tfidf_list:
    temp = []
    for key, val in l.items():
        temp.append(val)
    X.append(temp)

X = np.array(X)
##X = normalize(X)

scaler = MinMaxScaler()
X = scaler.fit(X).transform(X)

y = [ 0 for i in range(len(tokenized_docs))]
for i in range(int(len(tokenized_docs)/2)):
    y[i]=1
y = np.array(y)

##print(X)

skf = StratifiedKFold(n_splits=5, random_state=30, shuffle=True)
##skf.get_n_splits(X, y)
for train_index, test_index in skf.split(X, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    tclf = svm.LinearSVC().fit(X_train,y_train)
    predict = tclf.predict(X_test)
    
    print(predict)
    print(y_test)
    print(recall_score(y_test,predict),precision_score(y_test,predict),f1_score(y_test,predict))

[0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 1 0 0 1 0 1 0 0 0 1 1 0 1 0
 0 0 1 0 0 0 0 1 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.64 0.695652173913 0.666666666667
[1 0 0 1 1 0 0 0 1 1 0 1 0 1 0 0 1 1 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0
 0 1 0 0 1 0 0 0 0 0 1 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.52 0.722222222222 0.604651162791
[1 0 1 1 0 1 1 0 1 0 1 1 0 0 1 1 1 1 1 1 1 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 1
 0 1 0 0 0 0 1 1 0 1 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.72 0.75 0.734693877551
[1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 1 0 1 1 0 0 1 0 0 1 0 1
 1 1 1 1 0 0 0 0 0 0 1 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.8 0.645161290323 0.714285714286
[1 0 1 1 1 0 1 0 1 0 1 0 1 1 1 1 1 0 0 0 1 0 0 

In [253]:
## Create Emotion Dictionary
angry_dict = [
    'โทสะ','วิโรธ','ความโกรธ','ความฉุนเฉียว','ความขัดเคือง','ขัดแค้น','ขึ้งเคียด','ขึ้งโกรธ',
    'ดาลเดือด','ทำให้โกรธ','ทำให้ฉุนเฉียว','ทำให้เกิดโทสะ','ยั่วโมโห','โกรธ','ฉุนเฉียว','เคือง',
    'แค้น','เดือด','โมโห']
anticipate_dict = [
    'คาด','ตั้งตาคอย','หยั่งทราบ','คาดหมาย','มุ่งหวัง','ชิงทำ','ทำล่วงหน้า','คาดคิด','คาดหวัง',
    'คาดเดา','ตั้งตารอ','ประมาณการณ์','มาดหมาย','เป็นที่คาดหวัง']

In [201]:
def findDictFeature(dictionary):
    N = len(wordDictList)
    dict_feature = [0 for i in range(N)]
    for idx, wordDict in enumerate(wordDictList):
        for word, val in wordDict.items():
            if val > 0:
                if word in dictionary:
                    dict_feature[idx] = 1
                    print(idx,word)
    return np.array(dict_feature).reshape(N,1)

In [202]:
angry_dict_feature = findDictFeature(angry_dict)

25 โมโห
68 โกรธ
88 โกรธ
150 โกรธ


In [203]:
anticipate_dict_feature = findDictFeature(anticipate_dict)

161 คาด
163 คาดหวัง


In [220]:
Xnew = np.concatenate((X,angry_dict_feature,anticipate_dict_feature), axis=1)

In [274]:
##Xnew = normalize(Xnew)
##scaler = MinMaxScaler()
##Xnew = scaler.fit(Xnew).transform(Xnew)

adf = normalize(angry_dict_feature)
scaler = MinMaxScaler()
adf = scaler.fit(adf).transform(adf)

skf = StratifiedKFold(n_splits=5, random_state=30, shuffle=True)
##skf.get_n_splits(X, y)
for train_index, test_index in skf.split(adf, y):
    #print("TRAIN:", X[train_index], "TEST:", X[test_index])
    X_train, X_test = adf[train_index], adf[test_index]
    y_train, y_test = y[train_index], y[test_index]
    tclf = svm.LinearSVC().fit(X_train,y_train)
    predict = tclf.predict(X_test)
    
    print(predict)
    print(y_test)
    print(recall_score(y_test,predict),precision_score(y_test,predict),f1_score(y_test,predict))

[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.04 1.0 0.0769230769231
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.0 0.0 0.0
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.04 0.5 0.0740740740741
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.04 1.0 0.0769230769231
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0

/Users/nop/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/nop/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [273]:
angry_dict_feature.shape

(250, 1)

In [243]:
test = np.zeros(250).reshape(250,1)
for i in range (250):
    if i<125: 
        test[i]=1
    else: 
        test[i]=0
test.shape

(250, 1)

In [244]:
Xnew2 = np.concatenate((Xnew,test), axis=1)

In [275]:
scaler = MinMaxScaler()
Xnew = scaler.fit(Xnew).transform(Xnew)
skf2 = StratifiedKFold(n_splits=5, random_state=30, shuffle=True)

for train_index, test_index in skf.split(Xnew, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train2, X_test2 = Xnew[train_index], Xnew[test_index]
    y_train2, y_test2 = y[train_index], y[test_index]
    tclf = svm.LinearSVC().fit(X_train2,y_train2)
    predict2 = tclf.predict(X_test2)
    
    print(predict2)
    print(y_test2)
    print(recall_score(y_test2,predict2),precision_score(y_test2,predict2),f1_score(y_test2,predict2))

[0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 1 0 1 1 0 1 1 0 0 1 1 0 1 0 0 0 1 1 0 1 0
 0 0 1 1 0 0 0 1 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.56 0.608695652174 0.583333333333
[1 0 0 1 1 0 0 1 1 1 0 1 0 1 0 0 1 1 1 0 1 0 1 0 1 0 0 1 0 0 0 0 0 1 0 1 0
 1 1 0 0 1 0 0 0 0 0 1 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.56 0.666666666667 0.608695652174
[1 0 1 1 0 1 1 0 1 0 1 1 0 0 1 1 1 1 1 1 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1
 0 1 0 0 0 0 1 1 0 1 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.72 0.782608695652 0.75
[1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1 0 1 1 0 0 1 0 0 0 0 1
 1 1 1 1 0 0 0 0 0 0 1 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.8 0.666666666667 0.727272727273
[0 0 1 1 1 0 1 0 1 1 1 0 1 1 1 1 0 1 0 0 1 0 0 

In [246]:
Xnew2.shape

(250, 1147)